Upstream signaling network reconstruction
===============================

The following code use [pyPath library](https://github.com/saezlab/pypath) to reconstruct signaling network from a list of biological entities (csv format) and querying [OmnipathDB](http://omnipathdb.org/).

## 1. Load function

Import modules:

In [1]:
#!/usr/bin/env python
import csv
import time
import pypath
from pypath import curl
from pypath import data_formats

	:: No MySQL support.
	:: No MySQL support.
	:: Module `fabric` not available.
	:: Module `hcluster` not available.


In [2]:
def _upstream_signaling(pa, max_depth, to_be_explore, already_explored=[], current_depth=0, network_sif = []):
    """
    Param:
    pa: pypath env, 
    max_depth: maximum level of reconstruction, 
    to_be_explore: list of entities, 
    already_explored=[], 
    current_depth=0, 
    network_sif = []
    """
    
    # Stopping criteria 1
    if current_depth >= max_depth:
        print("Exploring alted due to maximum depth")
        return network_sif
    else:
        print('Exploration depth ' + str(current_depth))
    # Stopping criteria 2
    if len(to_be_explore) == 0:
        print("Exploring done")
        return(network_sif)
    # Start exploring
    new_to_be_explored = []
    for gene in to_be_explore:
        # get entity that get affected by MYC using vertex object (inhibition, stimulation or other)
        regulators_list = list(pa.gs_affects(gene))
        already_explored.append(gene)
        # get direction and sign of interation
        for reg in range(len(regulators_list)):
            # direction and sign
            edge = pa.get_edge(regulators_list[reg]["name"], gene)
            dirs = edge['dirs']
            sign_check = dirs.get_sign(dirs.reverse) # reverse: source ===> target
            # A pair of boolean values means if the interaction is stimulation and if it is inhibition, respectively [True, False] 
            if sign_check[0] == True and sign_check[1] == False:
                sign = 'stimulation'
            elif sign_check[0] == False and sign_check[1] == True: 
                sign = 'inhibition'
            elif sign_check[0] == True and sign_check[1] == True:
                sign = 'stimulation_and_inhibition'
            else:
                sign = 'unknown'
            if regulators_list[reg]["label"] not in already_explored:
                new_to_be_explored.append(regulators_list[reg]["label"])
            # ID, name, sign and provenance
            network_sif.append({"source_id":regulators_list[reg]["name"], "source_name":regulators_list[reg]["label"], \
                "provenance":list(regulators_list[reg]["sources"]), "target_name":gene, "sign":sign})
    current_depth += 1
    print("Depth explored")
    _upstream_signaling(pa, max_depth, new_to_be_explored, already_explored, current_depth, network_sif)
    return network_sif


def _print_to_csv(network, output_path):
    """
    Param: network, path of output file
    """
    f = open(output_path, "w+")
    # set headers
    f.write("source_id,source_name,target_name,sign,provenance\n")
    for e in network:
        f.write(e['source_id'] + ",")
        f.write(e['source_name'] + ",")
        f.write(e['target_name'] + ",")
        f.write(e['sign'] + ",")
        f.write(str(' '.join(e['provenance'])) + "\n")
    f.close()

def _get_stats(network, input_list, stats_output):
    """
    Param: network, list of input entities, output file path
    """
    f = open(stats_output, "w+")
    nodes = []
    edges = 0
    cov = 0
    for e in network:
        nodes.append(e['source_name'])
        nodes.append(e['target_name'])
        edges = edges + 1
    for i in input_list:
        if i in nodes:
            cov = cov + 1
    f.write("number of nodes,number of edges,input belonging to output,coverage\n")
    f.write(str(len(set(nodes))) + ",")
    f.write(str(edges) + ",")
    f.write(str(cov) + ",")
    f.write(str(cov*100/len(input_list)) + "\n")
    f.close()


## 2. Configuration

In [3]:
MAX_DEPTH = 8
INPUT_GENES = []
inputfile_path = 'input-910.csv'
outfile_path = 'md08-pypath_omnipathDB.csv'
stats_path = 'md010-stats.csv'

## 3. Read input file

In [4]:
with open(inputfile_path, 'rt') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        INPUT_GENES.append(row[0])

## 4. Main

In [5]:
# Init pypath
pa = pypath.PyPath()

# Load databases
with curl.cache_off():
   pa.load_resources(data_formats.pathway)



	=== d i s c l a i m e r ===

	All data coming with this module
	either as redistributed copy or downloaded using the
	programmatic interfaces included in the present module
	are available under public domain, are free to use at
	least for academic research or education purposes.
	Please be aware of the licences of all the datasets
	you use in your analysis, and please give appropriate
	credits for the original sources when you publish your
	results. To find out more about data sources please
	look at `pypath.descriptions` and
	`pypath.data_formats.urls`.

	> New session started,
	session ID: 'dr8gc'
	logfile: './log/dr8gc.log'
	pypath version: 0.7.120


	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/582ab5c0b5fffa42fb3e8d1757901ea1-`.
 > TRIP
	:: Loading data from cache previously downloaded from www.trpchannel.org
	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/fe747bcda0777e9f7cd8b1ab47db1538-show`.
		could not find uniprot for Gơq/11
		could not find uniprot for Ƣ-tubulin-5
		could not find uniprot for ơ-actinin
		could not find uniprot for ơ-actinin-1
		could not find uniprot for ơ1-syntrophin
		could not find uniprot for PLCƣ1
		could not find uniprot for VAMP
		could not find uniprot for Homer


	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/cb1a9773e75b45415423b000eb70a4a4-show`.
		could not find uniprot for TRPC2
	:: Loading data from cache previously downloaded from www.trpchannel.org
	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/76731af40f25e153c424547e40373728-show`.
		could not find uniprot for 14-3-3 protein Ƣ/ơ
		could not find uniprot for 14-3-3 protein ƥ
		could not find uniprot for 14-3-3 protein Ƨ
		could not find uniprot for ATP synthase Ƣ
		could not find uniprot for Gơq
		could not find uniprot for Gz-ơ
		could not find uniprot for NKAơ1
		could not find uniprot for NKAơ2
		could not find uniprot for NKAơ3
		could not find uniprot for ơSNAP
		could not find uni

	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/07a8b3ce8b01eb8c5b54e58fac2278f5-show`.
		could not find uniprot for Actin
		could not find uniprot for Akt
		could not find uniprot for AMPKơ2
		could not find uniprot for CaMKII
		could not find uniprot for Catenin-ơ2
		could not find uniprot for Catenin-Ƥ2
		could not find uniprot for Ƣ-arrestin 2
		could not find uniprot for Ƣ-tubulin-3
		could not find uniprot for KPNƢ1
		could not find uniprot for KvƢ2
		could not find uniprot for ơ-tubulin-1A
		could not find uniprot for ơ-tubulin-1B
		could not find uniprot for ơENaC
		could not find uniprot for PI3K-p85Ƣ
		could not find uniprot for PI3K-p85ơ
		could not find uniprot for PKARIIơ
		could not find uniprot for PKC
		could not find uniprot for PKCƥ
		could not find uniprot for PPARơ
		could not find uniprot for PRKD1/PKCƬ
		could not 

	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/227671676e74327da717392ec065629c-show`.
	:: Loading data from cache previously downloaded from www.trpchannel.org
	:: Ready. Resulted `plain text` of type unicode string.                                                                                              
	:: Local file at `/home/mlefebvre/.pypath/cache/00bdaab72698946e2c53a3ba1e161996-show`.


...

	:: Ready. Resulted `plain text` of type file object.                                                                                                 
	:: Local file at `/home/mlefebvre/.pypath/cache/2a72408fb2700d17cff8c9b48701de70-ncomms8866-s3.xlsx`.


        Processing attributes -- finished: 100%|██████████| 341/341 [00:00<00:00, 1.08kit/s]



 :: Comparing with reference lists... done.

 > 12687 interactions between 4351 nodes
 from 12 resources have been loaded,
 for details see the log: ./log/dr8gc.log
Exploration depth 0


        Setting directions -- finished: 100%|██████████| 12.7k/12.7k [00:18<00:00, 704it/s]


Depth explored
Exploration depth 1
Depth explored
Exploration depth 2
Depth explored
Exploration depth 3
Depth explored
Exploration depth 4
Depth explored
Exploration depth 5
Depth explored
Exploration depth 6
Depth explored
Exploration depth 7
Depth explored
Exploring alted due to maximum depth
--- Upstream signaling network in 39.32 seconds ---


In [6]:
start_time = time.time()
network = _upstream_signaling(pa, MAX_DEPTH, INPUT_GENES)
elapsed_time = round((time.time() - start_time), 2)
print("--- Upstream signaling network in %s seconds ---" % elapsed_time)
_print_to_csv(network, outfile_path)
_get_stats(network, INPUT_GENES, stats_path)

Exploration depth 0
Depth explored
Exploration depth 1
Exploring done
--- Upstream signaling network in 0.07 seconds ---
